In [6]:
####copied from course
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [2]:
def boyer_moore(p, p_bm, t):
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

 implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.

In [1]:
#naive matching algorithm - try all lineups of pattern against string
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1): # loop over alignments
        match = True
        num_alignments += 1
        for j in range(len(p)):          # loop over characters
            num_character_comparisons += 1
            if t[i + j] != p[j]:         # compare characters
                match = False            # mismatch; reject alignment
                break
        if match:
            occurrences.append(i)         # all chars matched; record
    return occurrences, num_alignments, num_character_comparisons

In [2]:
# example 1
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [3]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [4]:
def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if not p[j] == t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [7]:
# example 1
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [8]:
# example 2
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


In [ ]:
### the questions

In [35]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == ">":
                # rstrip removes trailing whitespace
                genome += line.rstrip()
    return genome

In [10]:
!"C:\Program Files (x86)\GnuWin32\bin\wget" --no-check-certificate http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GnuWin32/etc/wgetrc
--2022-02-10 14:32:05--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net... 108.138.225.105, 108.138.225.70, 108.138.225.59, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net|108.138.225.105|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: `chr1.GRCh38.excerpt.fasta'

     0K .......... .......... .......... .......... ..........  6%  589K 1s
    50K .......... .......... .......... .......... .......... 12% 1.34M 1s
   100K .......... .......... .......... .......... .......... 18% 1016K 1s
   150K .......... .......... .......... .......... .......... 25% 2.72M 1s
   200K .......... .......... .......... .......... .......... 31%  914K 1s
   250K .......... .......... .......... .......... .......... 37% 1.16M 0s
   300K .......... .......... .......

In [36]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [13]:
# questions 1 and 2
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, genome)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [14]:
# question 3
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, genome)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


## index assisted approximate matching
Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.  
Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.



In [1]:
import bisect

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [84]:
def indexed_approx_match(p, t, index, n):
    segment_length = round(len(p) / (n+1))
    all_matches = set()
    index_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        #print("Start,  end and query are {} {} {} \n".format(start, end, p[start:end]))
        matches = index.query(p[start:end])
        index_hits += len(matches)
        #print("Matches are {} \n".format(matches))
        for m in matches:
                if m < start or m-start+len(p) > len(t):
                    continue
                
                mismatches = 0
                for j in range(0,start):
                    if not p[j] == t[m-start+j]:
                        mismatches += 1
                        if mismatches > n:
                            break
            
                for j in range(end, len(p)):
                    if not p[j] == t[m-start+j]:
                        mismatches += 1
                        if mismatches > n:
                            break
                        
                if mismatches <= n:
                    if m - start not in all_matches:
                        all_matches.add(m - start)
                
    return sorted(list(all_matches)), index_hits

In [57]:
# kmers
k = 8
index = Index(genome, k)

In [85]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
matchie = indexed_approx_match(p, t, index, 2)
print(matchie)

([56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359], 90)


In [83]:
len(matchie)

19

In [36]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1): # loop over alignments
        match = True
        mismatches = 0
        for j in range(len(p)):          # loop over characters
            if t[i + j] != p[j]:         # compare characters
                mismatches += 1          # advance the number of mismatches
                if mismatches > 2:
                    match = False            # mismatch; reject alignment
                    break
        if match:
            occurrences.append(i)         # all chars matched; record
    return occurrences

In [37]:
gg = naive_2mm(p, genome)
print(gg)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


Question 6
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split \verb|ATATAT|ATATAT into two substring partitions, we would get partitions \verb|ATA|ATA (the first half) and \verb|TAT|TAT (second half).  But if we split \verb|ATATAT|ATATAT into two  subsequences  by taking every other character, we would get \verb|AAA|AAA (first, third and fifth characters) and \verb|TTT|TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as \verb|111222|111222, and splitting into two subsequences of every other character could be represented as \verb|121212|121212

In [2]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [87]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [88]:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


In [89]:
print(ind.query(p[1:]))

[1]


In [27]:
def query_subseq(p, t, index):
    # mismatches
    n = 2
    ispan = 2
    k = 8
    span = k + ((k - 1)*ispan)
    num_tries = len(p) - span + 1
    index_hits = 0
    all_matches = set()
    for i in range(num_tries):
        print("query string is {}\n".format(p[i:]))
        matches = index.query(p[i:])
        index_hits += len(matches)
        for m in matches:
            if m-i+len(p) > len(t):
                    continue
            mismatches = 0
            for j in range(len(p)):
                # print("pattern point: {}  pattern letter:{}  text point:{}  text letter:{}\n".format(j, p[j], m+j-i, t[m+j-i]))
                if not p[j] == t[m+j-i]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                if m - i not in all_matches:
                    all_matches.add(m - i)           
                    

    return sorted(list(all_matches)), index_hits
      

In [28]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
#occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
#matches = subseq_ind.query(p)

In [29]:
query_subseq(p, t, subseq_ind)

query string is to-morrow and to-morrow 

query string is o-morrow and to-morrow 

query string is -morrow and to-morrow 



([0, 14], 6)

In [30]:
t = open('pg1110.txt').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)

In [31]:
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)

query string is English measure backward

query string is nglish measure backward

query string is glish measure backward



In [32]:
print(occurrences)

[132576]


In [33]:
print(num_index_hits)

3


In [37]:
# q 6 - genome
subseq_ind = SubseqIndex(genome, 8, 3)

In [38]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
occurrences, num_index_hits = query_subseq(p, genome, subseq_ind)

query string is GGCGCGGTGGCTCACGCCTGTAAT

query string is GCGCGGTGGCTCACGCCTGTAAT

query string is CGCGGTGGCTCACGCCTGTAAT



In [44]:
print(num_index_hits)

79
